# Data analysis demo for H-core-M25 stellar hydro project
Last update: Nov 16, 2018.

This notebook contains a demonstration how to analyse the 3D filtered _moms_ data and the 1D radial profile _rprof_ data from _PPMstar_ 3D hydrodynamic simulations. 

### Data for this demo
The examples are for the project `H-core-M25` (this is the project identifier), the H-core convection simulations of a 25M$_\odot$ star. 

Two runs are used:
* `M29`: $768^3$ grid
* `M35`: $1536^3$ grid

`M29`, `M35` are the run identifier. Keep run and project identifier attached to all derived data products. 

Both runs have 1000x heating which increases their convective velocities by a factor of 10. 

For each run there are two types of data to be read for this demo:
* _moms_ data is the spatially filtered data (2-byte data on reduced grid by factor four in each direction) in 3D
* _rprofs_ data are spherically averaged radial profiles

### Location of data
The data is staged on the UVic Astrophysics Simulation Data Repository (ASDR) mounted in `/data/ASDR`. The repository contains the project folder `H-core-M25`. 

### Python asumptions

The server defaults each notebook to `%pylab ipympl`

In [1]:
## use this for final run to export with images to pdf, markdown or html
#%pylab inline

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as color
import nugridpy.utils as utils
import sys, os, time

# if you make changes to the ppmpy module (e.g. add your analysis methods via a pull 
# request) in the https://github.com/PPMstar/PyPPM repo you may want use that 
# updated version
#sys.path.insert(0,'/user/david/PyPPM/')
sys.path.insert(0,'/user/PyPPM/')
from ppmpy import ppm

cb = utils.linestylecb # colours

In [2]:
%%bash
ls /data/ASDR/H-core-M25/

M29-768
M35-1536


In [3]:
dir_repo    = '/data/ASDR'
dir_project = 'H-core-M25'
rprof = {}; moms = {}         # initialize dictionaries to hold rprof and moms instances

runs       = ['M29-768','M35-1536'] # select runs
moms_dumps = {}
moms_dumps[runs[0]] = 650       # select dump numbers for moms
moms_dumps[runs[1]] = 375   

#runs       = ['M29-768'] # select runs
#moms_dumps = [ 650     ] # select dump numbers for moms

# rprof instance holds radial profiles for all dumps
# moms instance holds only one dump at a time
for run in runs:        
    path = os.path.join(dir_repo,dir_project,run)
    # radial profile:
    rprof[run] = ppm.RprofSet(os.path.join(path,'rprofs'))
    moms[run] = ppm.MomsDataSet(os.path.join(path,'myavsbq'),moms_dumps[run])
print("moms and rprof dictionary created")

51 rprof files found in '/data/ASDR/H-core-M25/M29-768/rprofs/.
Dump numbers range from 650 to 700.
Reading history file '/data/ASDR/H-core-M25/M29-768/rprofs/HcoreE00768-0000.hstry'.
51 .aaa files found in '/data/ASDR/H-core-M25/M29-768/myavsbq/.
Dump numbers range from 650 to 700.
The PPMstar grid is being constructed, this can take a moment
51 rprof files found in '/data/ASDR/H-core-M25/M35-1536/rprofs/.
Dump numbers range from 375 to 425.
Reading history file '/data/ASDR/H-core-M25/M35-1536/rprofs/HcoreE01536-0000.hstry'.
51 .aaa files found in '/data/ASDR/H-core-M25/M35-1536/myavsbq/.
Dump numbers range from 375 to 425.
The PPMstar grid is being constructed, this can take a moment
moms and rprof dictionary created


In [4]:
# get info about moms instance
# help(moms['M29-768'])

## Basic grid properties

In [5]:
x,y,z,r=moms['M29-768'].get_grid()

In [6]:
print(192**3,len(r))

7077888 7077888


In [7]:
print("Distance center of grid to max x value of domain: %6.4f Mm" % moms['M35-1536'].get_grid()[0][-1])

Distance center of grid to max x value of domain: 2493.4895 Mm


### Histogram of radii
* increasing to 1/2 length of grid, then decreasing as only fraction of shell in box

In [8]:
ifig=0
ifig += 1; plt.close(ifig);  plt.figure(ifig)
hist(r,86)
xmax = moms['M35-1536'].get_grid()[0][-1]
vlines(xmax,0,2.3e5,linestyles='--',lw=0.5)
xlabel('radius / Mm')
ylabel('')

FigureCanvasNbAgg()

Text(0, 0.5, '')

### Some more experiments with coordinates

In [9]:
xx=reshape(x,[192,192,192])

In [10]:
ifig += 1; plt.close(ifig);  plt.figure(ifig)
#plot(xx[0][0])
plot(x[0:1000])

FigureCanvasNbAgg()

### Planar slice image

In [11]:
# grab the grid
x,y,z,r = moms['M35-1536'].get_grid()

# they are flattened arrays, rearrange
resolution = moms['M35-1536'].momsdata.resolution
r_matrix = np.reshape(r,(resolution,resolution,resolution))

# extent x,y
extent=[min(x),max(x),min(y),max(y)]

# slice number
slice_num = 200

#### Radius

In [12]:
ifig += 1; plt.close(ifig); plt.figure(ifig)
plt.imshow(r_matrix[:][:][slice_num],extent=extent)
plt.ylabel('y')
plt.xlabel('x')
cbar = plt.colorbar()

# label colorbar
cbar.ax.set_ylabel('radius')

FigureCanvasNbAgg()

DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7f378d69fbe0>
DEBUG:matplotlib.colorbar:Using auto colorbar locator on colorbar
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7f378d69fbe0>
DEBUG:matplotlib.colorbar:Setting pcolormesh


Text(0, 0.5, 'radius')

### Properties of the grid and time resolution

In [13]:
# spatial resolution for 768 momsdata
print('The spatial resolution of 768 momsdata is {:0.3f}'.format(np.diff(moms['M29-768'].get_grid()[0])[0]),'Mm')
print('While PPMStar 768 has a spatial resolution of {:0.3f}'.format(np.diff(moms['M29-768'].get_grid()[0])[0]/4.),'Mm')

print('')

# spatial resolution for 1536 momsdata
print('The spatial resolution of 1536 momsdata is {:0.3f}'.format(np.diff(moms['M35-1536'].get_grid()[0])[0]),'Mm')
print('While PPMStar 1536 has a spatial resolution of {:0.3f}'.format(np.diff(moms['M35-1536'].get_grid()[0])[0]/4.),'Mm')

print('')

# what is the extent of the simulation?
print('The extent of the simulation is then {:0.0f}'.format(np.diff(moms['M35-1536'].get_grid()[0])[0]/4. * 1536/2.),'Mm')

The spatial resolution of 768 momsdata is 26.042 Mm
While PPMStar 768 has a spatial resolution of 6.510 Mm

The spatial resolution of 1536 momsdata is 13.021 Mm
While PPMStar 1536 has a spatial resolution of 3.255 Mm

The extent of the simulation is then 2500 Mm


In [14]:
#  for 768 momsdata
print('The temporal resolution of 768 and 1536 momsdata is the same as the PPMStar output which averages around {:0.2f}'.
      format(np.mean(np.diff(rprof['M29-768'].get_history().get('time(mins)')))),'minutes per dump')

print('')

print('The run-time temporal resolution of the PPMStar output averages around {:0.2f}'.
      format(np.mean(rprof['M35-1536'].get_history().get('dt(secs)'))),'seconds per cycle')

The temporal resolution of 768 and 1536 momsdata is the same as the PPMStar output which averages around 167.19 minutes per dump

The run-time temporal resolution of the PPMStar output averages around 2.73 seconds per cycle


## Find Times For Dumps

As hinted at in the above section, there is a history file that gives us information about the run. This is located in the rprof files themselves

In [15]:
# get the simulation time in seconds for dump 100 in the 768 and 1536 runs
print('{:0.1f} seconds '.format(rprof['M29-768'].get_history().get('time(secs)')[moms_dumps['M29-768']]) + 
'have passed since the simulation started for the 768 run')
print('{:0.1f} seconds '.format(rprof['M35-1536'].get_history().get('time(secs)')[moms_dumps['M35-1536']]) +
      'have passed since the simulation started for the 1536 run')

6507520.0 seconds have passed since the simulation started for the 768 run
3756320.0 seconds have passed since the simulation started for the 1536 run


## What quantities have what index?

The following quantities written into the moms data file which can be called with an index:

| index | quantity            |
|:-----:|:--------------:     |
| 0     | x                   |
| 1     | $\vec{u_{x}}$                  |   
| 2     | $\vec{u_{y}}$             | 
|  3    | $\vec{u_{z}}$             | 
|  4    | $\lvert\vec{u_{t}}\rvert$         | 
|  5    | $\lvert\vec{u_{r}}\rvert$             | 
|  6    | $\lvert\vec{\omega}\rvert$      | 
|  7    | P              | 
|  8    | rho            | 
|  9    | fv             | 

* Note that these are just 10 out of 32 quantities that can be made available in the moms data. 
* fv is the fractional volume of the material initially only outside the convection zone.

#### Some Helpful Definitions

$\mu$ = fv $\times$ 0.617 + (1 - fv) $\times$ 0.669

T = $\frac{P \mu}{\rho R_{gas}}$

$R_{gas}$ = 8.314462

$\vec{\omega}$ = $\vec{\nabla} \times \vec{u}$

## Radial profiles
Radial profiles can be taken from the _rprof_ data sets. They can also be constructed from the _moms_ data. This is demonstrated below. 

In [16]:
# define variables for dump number, rprof and moms

runid = 'M29-768'   # select run id for the rest of the notebook
runid = 'M35-1536'
thisdump = moms_dumps[runid]
thisrprof = rprof[runid]
thismoms = moms[runid]

# get T9 and Ut
P_rprof = thisrprof.get('P0',fname=thisdump,resolution='h')[0::2] + thisrprof.get('P1',fname=thisdump,resolution='h')[0::2]
rho_rprof = thisrprof.get('Rho0',fname=thisdump,resolution='h')[0::2] + thisrprof.get('Rho1',fname=thisdump,resolution='h')[0::2]
FV_rprof = thisrprof.get('FV',fname=thisdump,resolution='h')[0::2]

# T9 in rprof class is not correct, calculate directly
T9_rprof = P_rprof * (0.617*FV_rprof + 0.669*(1-FV_rprof)) / (8.314462 * rho_rprof)

R_rprof = thisrprof.get('R',fname=thisdump,resolution='l')
Ut_rprof = thisrprof.get('|Ut|',fname=thisdump)

In [17]:

# make an rprof of temperature and ut
ut_avg, radial_axis = thismoms.get_rprof(4,thisdump)

# first we need to construct T from quantities
mu = 0.617 * thismoms.get(9,fname=thisdump) + (1 - thismoms.get(9,fname=thisdump))*0.669
P = thismoms.get(7,fname=thisdump)
rho = thismoms.get(8,fname=thisdump)
Rgas = 8.314462

# put it all together
T = (mu * P) / (Rgas * rho)

# we can give the rprof method an array to be spherically averaged
T_avg, radial_axis = thismoms.get_rprof(T,thisdump)

/usr/local/lib/python3.6/dist-packages/scipy/stats/_binned_statistic.py:607: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = result[core]


In [18]:
# plot
ifig += 1; plt.close(ifig);  plt.figure(ifig)

plt.plot(R_rprof,T9_rprof,label='Rprof',ls=cb(0)[0],color=cb(0)[2])
plt.plot(radial_axis,T_avg,label='Moms',ls=cb(1)[0],color=cb(1)[2])
plt.xlabel('R')
plt.ylabel('T9')

plt.legend()

FigureCanvasNbAgg()

In [19]:
# plot
ifig += 1; plt.close(ifig);  plt.figure(ifig)

plt.plot(R_rprof,Ut_rprof,label='Rprof',ls=cb(0)[0],color=cb(0)[2])
plt.plot(radial_axis,ut_avg,label='Moms',ls=cb(1)[0],color=cb(1)[2])

plt.xlabel('R')
plt.ylabel('|Ut|')

plt.legend()

FigureCanvasNbAgg()

### Planar Slice Image

In [20]:
x,y,z,r = moms[runid].get_grid()

# they are flattened arrays, rearrange
resolution = moms[runid].momsdata.resolution
r_matrix = np.reshape(r,(resolution,resolution,resolution))

# extent x,y
extent=[min(x),max(x),min(y),max(y)]

# slice number
slice_num = int(resolution/2)

#### T9

In [21]:
T_matrix = np.reshape(T,(resolution,resolution,resolution))

In [22]:
ifig += 1; plt.close(ifig);  plt.figure(ifig)

plt.imshow(T_matrix[:][:][slice_num],extent=extent)
plt.ylabel('y')
plt.xlabel('x')
cbar = plt.colorbar()

# label colorbar
cbar.ax.set_ylabel('T9')

FigureCanvasNbAgg()

DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7f378d4c3ef0>
DEBUG:matplotlib.colorbar:Using auto colorbar locator on colorbar
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7f378d4c3ef0>
DEBUG:matplotlib.colorbar:Setting pcolormesh


Text(0, 0.5, 'T9')

#### |Ut|

In [23]:
ut_matrix = np.reshape(thismoms.get(4,thisdump),(resolution,resolution,resolution))

In [24]:
ifig += 1; plt.close(ifig); plt.figure(ifig)
plt.imshow(ut_matrix[:][:][slice_num],extent=extent)
plt.ylabel('y')
plt.xlabel('x')
cbar = plt.colorbar()

# label colorbar
cbar.ax.set_ylabel('|Ut|')

FigureCanvasNbAgg()

DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7f378d45edd8>
DEBUG:matplotlib.colorbar:Using auto colorbar locator on colorbar
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7f378d45edd8>
DEBUG:matplotlib.colorbar:Setting pcolormesh


Text(0, 0.5, '|Ut|')

## FV Colourmap of a Plane (x=y=0)

In [25]:
# x=y=0, a particular z slice. convert to an 8-bit number
fv = np.reshape(thismoms.get(9,thisdump),(resolution,resolution,resolution))
fv_bit = 251 + 13.35455532 * np.log(fv[:][:][96])

In [26]:
FV_cmap_str = '''
Anot: 0 0.0       
Anot: 18 0.1058824
Anot: 56 0.2745098
Anot: 75 0.7843137
Anot: 123 1.0 
Anot: 158 1.0
Anot: 184 0.5490196
Anot: 203 0.454902
Anot: 255 0.1254902
Cnot: 0 0.0 0.0 0.0           
Cnot: 48 0.0 0.0 0.2509804    
Cnot: 56 0.0 0.2352941 0.627451
Cnot: 65 0.0 0.7843137 1.0   
Cnot: 75 1.0 1.0 1.0         
Cnot: 100 1.0 1.0 0.0         
Cnot: 186 1.0 0.0 0.0        
Cnot: 244 0.5019608 0.0 0.0   
Cnot: 255 0.5019608 0.0 0.0   
'''
cmap = ppm.colourmap_from_str(FV_cmap_str, segment=(5, 251))

# normalize to our 255 bit range
norm = mpl.colors.Normalize(vmin=5, vmax=251)

#### Square Image

In [27]:
my_dpi = 300
ifig+=1; plt.close(ifig); plt.figure(ifig,figsize=(536/my_dpi, 536/my_dpi), dpi=my_dpi)
x,y,z,r = thismoms.get_grid()
plt.pcolor(np.unique(x),np.unique(y),fv_bit,cmap=cmap,norm=norm)

plt.axis('off')

FigureCanvasNbAgg()

(-2493.489501953125, 2493.489501953125, -2493.489501953125, 2493.489501953125)